# Applied Machine Learning: Module 4 (Supervised Learning, Part II)## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [9]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

def blight_model():
    us_states  = ['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL',
             'IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT',
             'NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI',
             'SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']
    disposition_replace = {
        'Responsible by Default': 'By default',
        'Responsible by Determination': 'By determination', 
        'Responsible (Fine Waived) by Deter': 'Fine Waived',
        'Responsible by Admission': 'By admission',
        'SET-ASIDE (PENDING JUDGMENT)': 'Pending',
        'PENDING JUDGMENT': 'Pending',
        'Not responsible by Dismissal': 'Not responsible',
        'Not responsible by City Dismissal': 'Not responsible',
        'Not responsible by Determination': 'Not responsible',
        'Responsible (Fine Waived) by Admis': 'Fine Waived',
        'Responsible - Compl/Adj by Default': 'By default',
        'Responsible - Compl/Adj by Determi': 'By determination',
        'Responsible by Dismissal': 'By default'
    }

    def clean(DataSet):
        # Collapse state data into in-state or out of state (or out of country)
        DataSet['in_state'] = DataSet.state.apply(lambda x: 'yes' if x=='MI' else ('no' if x in us_states else 'foreign'))

        # Clean disposition values
        DataSet.disposition.replace(disposition_replace, inplace=True)

        # These columns just aren't useful to us
        DataSet.drop([
            'violation_zip_code', 
            'non_us_str_code', 
            'grafitti_status', 
            'violator_name', 
            'violation_street_number', 
            'violation_street_name', 
            'violation_description',
            'mailing_address_str_number', 
            'mailing_address_str_name',
            'city',
            'state',
            'zip_code',
            'country',
            'clean_up_cost'
        ], axis=1, inplace=True)
        
            # Date columns
        DataSet['ticket_issued_date'] = pd.to_datetime(DataSet['ticket_issued_date'])
        DataSet['hearing_date'] = pd.to_datetime(DataSet['hearing_date'])
        DataSet['hearing_date'].fillna((DataSet.hearing_date - DataSet.hearing_date.min()).mean() + DataSet.hearing_date.min(), inplace=True)
        DataSet['ticket_issued_date'].fillna((DataSet.ticket_issued_date - DataSet.ticket_issued_date.min()).mean() + DataSet.ticket_issued_date.min(), inplace=True)
        DataSet['date_gap'] = ((DataSet['hearing_date'] - DataSet['ticket_issued_date'])/np.timedelta64(1, 'D')).astype(int)

        # Set index
        DataSet.set_index('ticket_id', inplace=True)
        
        return DataSet
    
    def make_features(DataSet):
        dummy = pd.get_dummies(DataSet[['in_state', 'disposition', 'agency_name']])

        # Make sure all the categorical columns got created, even if they're blank
        # This ensures training and test data have the same columns
        for cat_col in ['in_state_yes', 'in_state_no', 'in_state_foreign',
                        'disposition_By admission', 'disposition_By default', 'disposition_By determination', 'disposition_Fine Waived',
                        'agency_name_Buildings, Safety Engineering & Env Department',
                        'agency_name_Department of Public Works',
                        'agency_name_Detroit Police Department',
                        'agency_name_Health Department'
                       ]:
            if not cat_col in dummy.columns:
                dummy[cat_col] = 0

        return pd.concat(
            (DataSet[['discount_amount', 'judgment_amount', 'date_gap']], 
             dummy), 
            axis=1)
    
    DS = clean(pd.read_csv('readonly/train.csv',encoding = "ISO-8859-1"))
    DS.dropna(subset=['compliance'], inplace=True)
    DS.compliance = DS.compliance.astype("category")
    y_train = DS['compliance']
    X_train = make_features(DS)
    X_train.drop('agency_name_Neighborhood City Halls', axis=1, inplace=True)
        
    #X_train, X_test1, y_train, y_test1 = train_test_split(X_train, y_train, random_state = 0)
    
    clf = GradientBoostingClassifier()
    
    GS = GridSearchCV(clf, {'learning_rate': [0.005, 0.1, 0.3, 1],'max_depth':[2, 3, 5]},scoring='roc_auc', cv=5)
    GS.fit(X_train, y_train)

    #gbc = GradientBoostingClassifier(learning_rate=0.1, max_depth=3).fit(X_train, y_train)
    
    X_test = make_features(clean(pd.read_csv('readonly/test.csv',encoding = "ISO-8859-1")))
    X_test['compliance'] = GS.predict_proba(X_test)[:, 0].astype('float32')
    g=print("Best score: {}".format(GS.best_score_))
    return X_test['compliance'], g

In [10]:
blight_model()

C:\Users\mario\Anaconda3\envs\actumlogos-gpu\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (11,12,31) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Best score: 0.7893503257088573


(ticket_id
 284932    0.939389
 285362    0.982939
 285361    0.931829
 285338    0.939389
 285346    0.931829
             ...   
 376496    0.982659
 376497    0.982659
 376499    0.935923
 376500    0.935923
 369851    0.773606
 Name: compliance, Length: 61001, dtype: float32,
 None)

In [95]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

def blight_model1():
    def cleantrain(DataSet):
        # These columns just aren't useful to us
        DataSet.drop(['agency_name','inspector_name','violator_name', 'violation_street_number', 
                 'violation_street_name', 'violation_zip_code', 'mailing_address_str_number', 
                 'mailing_address_str_name', 'zip_code', 'non_us_str_code', 
                 'violation_code', 'violation_description', 'disposition','clean_up_cost',
                 'payment_amount', 'balance_due', 'payment_date', 'payment_status',
                 'collection_status', 'grafitti_status', 'compliance_detail',
                 'fine_amount', 'admin_fee', 'state_fee', 'late_fee'], axis=1, inplace=True)
        
            # Date columns
        DataSet['ticket_issued_date'] = pd.to_datetime(DataSet['ticket_issued_date'])
        DataSet['hearing_date'] = pd.to_datetime(DataSet['hearing_date'])
        DataSet['hearing_date'].fillna((DataSet.hearing_date - DataSet.hearing_date.min()).mean() + DataSet.hearing_date.min(), inplace=True)
        DataSet['ticket_issued_date'].fillna((DataSet.ticket_issued_date - DataSet.ticket_issued_date.min()).mean() + DataSet.ticket_issued_date.min(), inplace=True)
        DataSet['date_gap'] = ((DataSet['hearing_date'] - DataSet['ticket_issued_date'])/np.timedelta64(1, 'D')).astype(int)

        # Verificar si viven en Detroit, en otra Ciudad, otro estado o en otro País
        DataSet['state'] = DataSet.state.apply(lambda x: 1 if x=='MI' else 0)
        DataSet['city'] = DataSet.city.apply(lambda x: 1 if x=='Detroit' or x== 'DETROIT' or x== 'det' or x== 'Det' or x== 'detroit' else 0)
        DataSet['country'] = DataSet.country.apply(lambda x: 1 if x=='USA' else 0)
        
        # Set index
        DataSet.set_index('ticket_id', inplace=True)
        
        return DataSet
    def cleantest(DataSet):
        # These columns just aren't useful to us
        DataSet.drop(['agency_name','inspector_name','violator_name', 'violation_street_number', 
                 'violation_street_name', 'violation_zip_code', 'mailing_address_str_number', 
                 'mailing_address_str_name', 'zip_code', 'non_us_str_code', 
                 'violation_code', 'violation_description', 'disposition','clean_up_cost',
                 'grafitti_status', 'fine_amount', 'admin_fee', 'state_fee', 'late_fee'], axis=1, inplace=True)
        
            # Date columns
        DataSet['ticket_issued_date'] = pd.to_datetime(DataSet['ticket_issued_date'])
        DataSet['hearing_date'] = pd.to_datetime(DataSet['hearing_date'])
        DataSet['hearing_date'].fillna((DataSet.hearing_date - DataSet.hearing_date.min()).mean() + DataSet.hearing_date.min(), inplace=True)
        DataSet['ticket_issued_date'].fillna((DataSet.ticket_issued_date - DataSet.ticket_issued_date.min()).mean() + DataSet.ticket_issued_date.min(), inplace=True)
        DataSet['date_gap'] = ((DataSet['hearing_date'] - DataSet['ticket_issued_date'])/np.timedelta64(1, 'D')).astype(int)

        # Verificar si viven en Detroit, en otra Ciudad, otro estado o en otro País
        DataSet['state'] = DataSet.state.apply(lambda x: 1 if x=='MI' else 0)
        DataSet['city'] = DataSet.city.apply(lambda x: 1 if x=='Detroit' or x== 'DETROIT' or x== 'det' or x== 'Det' or x== 'detroit' else 0)
        DataSet['country'] = DataSet.country.apply(lambda x: 1 if x=='USA' else 0)
        
        # Set index
        DataSet.set_index('ticket_id', inplace=True)
        
        return DataSet

    Train = cleantrain(pd.read_csv('readonly/train.csv',encoding = "ISO-8859-1"))
    Test = cleantest(pd.read_csv('readonly/test.csv',encoding = "ISO-8859-1"))
    Train.dropna(subset=['compliance'], inplace=True)
    #Train.compliance = Train.compliance.astype("category")
    y_train = Train['compliance']
    #X_train = Train[['city', 'state', 'country', 'ticket_issued_date', 'hearing_date', 'discount_amount','judgment_amount','date_gap']].astype("category")
    X_train = Train[['city', 'state', 'country', 'discount_amount','judgment_amount','date_gap']].astype("category")
    X_test = Test[['city', 'state', 'country', 'discount_amount','judgment_amount','date_gap']].astype("category")
    
    clf = GradientBoostingClassifier()
    #clf = RandomForestClassifier()
    #clf.fit(X_train, y_train)
    GS = GridSearchCV(clf, {'learning_rate': [0.005, 0.1, 0.2, 0.3, 1, 5],'max_depth':[2, 3, 4, 5]},scoring='roc_auc', cv=5)
    GS.fit(X_train, y_train)
    
    X_test['compliance'] = GS.predict_proba(X_test)[:, 0].astype('float32')
    #X_test['compliance'] = clf.predict_proba(X_test).astype('float32')
    g=print("Best score: {}".format(GS.best_score_))

    return X_test['compliance'], g

In [96]:
blight_model1()

C:\Users\mario\Anaconda3\envs\actumlogos-gpu\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (11,12,31) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Best score: 0.7534492712767691


(ticket_id
 284932    0.933555
 285362    0.971118
 285361    0.916104
 285338    0.927873
 285346    0.908327
             ...   
 376496    0.969042
 376497    0.969042
 376499    0.916104
 376500    0.916104
 369851    0.733236
 Name: compliance, Length: 61001, dtype: float32,
 None)

In [93]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

def blight_mode2():
    def cleantrain(DataSet):
        # These columns just aren't useful to us
        DataSet.drop(['agency_name','inspector_name','violator_name', 'violation_street_number', 
                 'violation_street_name', 'violation_zip_code', 'mailing_address_str_number', 
                 'mailing_address_str_name', 'zip_code', 'non_us_str_code', 
                 'violation_code', 'violation_description', 'disposition','clean_up_cost',
                 'payment_amount', 'balance_due', 'payment_date', 'payment_status',
                 'collection_status', 'grafitti_status', 'compliance_detail',
                 'fine_amount', 'admin_fee', 'state_fee', 'late_fee'], axis=1, inplace=True)
        
            # Date columns
        DataSet['ticket_issued_date'] = pd.to_datetime(DataSet['ticket_issued_date'])
        DataSet['hearing_date'] = pd.to_datetime(DataSet['hearing_date'])
        DataSet['hearing_date'].fillna((DataSet.hearing_date - DataSet.hearing_date.min()).mean() + DataSet.hearing_date.min(), inplace=True)
        DataSet['ticket_issued_date'].fillna((DataSet.ticket_issued_date - DataSet.ticket_issued_date.min()).mean() + DataSet.ticket_issued_date.min(), inplace=True)
        DataSet['date_gap'] = ((DataSet['hearing_date'] - DataSet['ticket_issued_date'])/np.timedelta64(1, 'D')).astype(int)

        # Verificar si viven en Detroit, en otra Ciudad, otro estado o en otro País
        DataSet['state'] = DataSet.state.apply(lambda x: 1 if x=='MI' else 0)
        DataSet['city'] = DataSet.city.apply(lambda x: 1 if x=='Detroit' or x== 'DETROIT' or x== 'det' or x== 'Det' or x== 'detroit' else 0)
        DataSet['country'] = DataSet.country.apply(lambda x: 1 if x=='USA' else 0)
        
        DataSet.drop(['ticket_issued_date', 'hearing_date'], axis=1, inplace=True)
        
        # Set index
        DataSet.set_index('ticket_id', inplace=True)
        
        return DataSet
    def cleantest(DataSet):
        # These columns just aren't useful to us
        DataSet.drop(['agency_name','inspector_name','violator_name', 'violation_street_number', 
                 'violation_street_name', 'violation_zip_code', 'mailing_address_str_number', 
                 'mailing_address_str_name', 'zip_code', 'non_us_str_code', 
                 'violation_code', 'violation_description', 'disposition','clean_up_cost',
                 'grafitti_status', 'fine_amount', 'admin_fee', 'state_fee', 'late_fee'], axis=1, inplace=True)
        
            # Date columns
        DataSet['ticket_issued_date'] = pd.to_datetime(DataSet['ticket_issued_date'])
        DataSet['hearing_date'] = pd.to_datetime(DataSet['hearing_date'])
        DataSet['hearing_date'].fillna((DataSet.hearing_date - DataSet.hearing_date.min()).mean() + DataSet.hearing_date.min(), inplace=True)
        DataSet['ticket_issued_date'].fillna((DataSet.ticket_issued_date - DataSet.ticket_issued_date.min()).mean() + DataSet.ticket_issued_date.min(), inplace=True)
        DataSet['date_gap'] = ((DataSet['hearing_date'] - DataSet['ticket_issued_date'])/np.timedelta64(1, 'D')).astype(int)
        
        # Verificar si viven en Detroit, en otra Ciudad, otro estado o en otro País
        DataSet['state'] = DataSet.state.apply(lambda x: 1 if x=='MI' else 0)
        DataSet['city'] = DataSet.city.apply(lambda x: 1 if x=='Detroit' or x== 'DETROIT' or x== 'det' or x== 'Det' or x== 'detroit' else 0)
        DataSet['country'] = DataSet.country.apply(lambda x: 1 if x=='USA' else 0)
        
        
        
        # Set index
        DataSet.set_index('ticket_id', inplace=True)
        
        return DataSet

    Train = cleantrain(pd.read_csv('readonly/train.csv',encoding = "ISO-8859-1"))
    Test = cleantest(pd.read_csv('readonly/test.csv',encoding = "ISO-8859-1"))
    Train.dropna(subset=['compliance'], inplace=True)
    #Train.compliance = Train.compliance.astype("category")
    y_train = Train['compliance']
    X_train = Train[['city', 'state', 'country', 'discount_amount','judgment_amount','date_gap']]#.astype("category")
    X_test = Test[['city', 'state', 'country', 'discount_amount','judgment_amount','date_gap']]#.astype("category")
    
    
    clf = RandomForestClassifier()
    GS = GridSearchCV(clf, {'n_estimators': range(70, 150, 3)}, scoring='roc_auc', cv=5).fit(X_train, y_train)
    #clf = GradientBoostingClassifier()
    #GS = GridSearchCV(clf, {'learning_rate': [0.005, 0.1, 0.2, 0.3, 1, 5],'max_depth':[2, 3, 4, 5]},scoring='roc_auc', cv=5)
    #GS.fit(X_train, y_train)
    
    X_test['compliance'] = GS.predict_proba(X_test)[:, 0].astype('float32')
    #X_test['compliance'] = clf.predict_proba(X_test).astype('float32')
    g=print("Best score: {}".format(GS.best_score_))

    return X_test['compliance'], g

In [94]:
blight_mode2()

C:\Users\mario\Anaconda3\envs\actumlogos-gpu\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (11,12,31) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Best score: 0.6981406989289127


C:\Users\mario\Anaconda3\envs\actumlogos-gpu\lib\site-packages\ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(ticket_id
 284932    0.906672
 285362    1.000000
 285361    0.916503
 285338    1.000000
 285346    0.939460
             ...   
 376496    1.000000
 376497    1.000000
 376499    0.936761
 376500    0.936761
 369851    0.776987
 Name: compliance, Length: 61001, dtype: float32,
 None)